# Clean Slate: Estimating offenses eligible for expungement under varying conditions
> Prepared by [Laura Feeney](https://github.com/laurafeeney) for Code for Boston's [Clean Slate project](https://github.com/codeforboston/clean-slate).

## Purpose & Notes
Clean and merge Middlesex DA data with expungement eligibility data. 
This data was sourced from the Middlesex DA website: https://www.middlesexda.com/public-information/pages/prosecution-data-and-statistics

Description from website: "The following is data from our Damion Case Management System pertaining to prosecution statistics for the time period from January 1, 2014, through January 1, 2020."

The download is available as an Excel file. Opening excel in Python was too slow, so I manually converted it to csv, and imported via csv.

Note: This did not have age or DOB. 

The Middlesex DA site says this should be prosecutions for 2014 - 2019. However, not all offense dates nor all disposition dates are within this timeline.

### Merging
The Charges have a slightly different format, character set, and phrasing of charges than the Suffolk and NW data, which we obtained first. Will need some manual updating to get expungment info for all charges.

### Still to do
* clean up sex and murder columns -- needs to be filled in for the unmatched from PCD / MCL / extra info. 

-----

In [1]:
import pandas as pd
pd.set_option("display.max_rows", 200)
import numpy as np
import regex as re
import glob, os
import datetime 
from datetime import date 

#print(os.getcwd())
os.chdir("../../data/raw")
#print(os.getcwd())

In [2]:
#ms_raw = pd.read_excel('damion_database_2014-2019_6.30.xlsx') # too slow to do this way
ms_raw = pd.read_csv('damion_database_2014-2019_6.30.csv') 
columns = ['Case Number', 'Offense Date', 'Date of Filing', 'Court Location', 
           'Charge/Crime Code', 'Charge/Crime Description', 'Charge/Crime Type',
           'Disposition Description', 'Disposition Date']
ms = ms_raw[columns].replace()
ms.head()

,Case Number,Offense Date,Date of Filing,Court Location,Charge/Crime Code,Charge/Crime Description,Charge/Crime Type,Disposition Description,Disposition Date
0,14-01-479818,12/30/2013,1/2/2014,SOM,90/23/D,"LICENSE SUSPENDED, OP MV WITH c90 §23",Drugs/Distribution/Possession with Intent,GUILTY FILED,6/2/2014
1,14-01-479818,12/30/2013,1/2/2014,SOM,94C/32C/C,"DRUG, POSSESS TO DISTRIB CLASS D c94C §32C(a)",Drugs/Distribution/Possession with Intent,NOLLE PROSEQUI,6/2/2014
2,14-01-479819,12/31/2013,1/2/2014,SOM,94C/34/C,"DRUG, POSSESS CLASS B c94C §34",Drugs/Possession,DISMISSED W/O PREJUDICE,5/9/2014
3,14-01-479819,12/31/2013,1/2/2014,SOM,90/17/A,SPEEDING * c90 §17,Drugs/Possession,RESPONSIBLE,5/9/2014
4,14-01-479819,12/31/2013,1/2/2014,SOM,89/4A,MARKED LANES VIOLATION * c89 §4A,Drugs/Possession,RESPONSIBLE,5/9/2014


### Cleaning and variable prep

In [3]:
ms.rename(columns={"Charge/Crime Description":"Charge"}, inplace=True)

# Label CMR offenses (Code of Mass Regulations)
ms['CMRoffense'] = False
ms.loc[ms['Charge'].str.contains("CMR"), 'CMRoffense'] = True

#Extract Chapter, Section, and Paragrah (I think the third one would be paragraph? It isn't always populated)
chsec = ms['Charge/Crime Code'].str.split("/", n = 2, expand = True) 
ms['Chapter'] = chsec[0]
ms['Section'] = chsec[1]
ms['Paragraph'] = chsec[2]

# Remove weird A character, and create a version with no spaces and no extra characters. This file has different spacing 
# than NW and Suffolk or Master Crime List descriptions.

ms['Charge_noA'] = ms['Charge'].map(lambda x: x.replace('Â',""))
ms['Charge_alnum'] = ms['Charge_noA'].str.replace(r'\W+', '', )
ms['Charge_alnum2'] = ms['Charge_alnum'].str.replace("[^a-zA-Z0-9 -]","",)

# Proxy for age -- using a juvenile court
ms['JuvenileC'] =  False
ms.loc[ms['Court Location'].str.contains("JU"), 'JuvenileC'] = True

### dates. Supposed to be 2014-2019

In [4]:
reference_date = datetime.date(2020, 9, 1) # using "today.date() wouldn't be stable"

ms['Offense Date'] = pd.to_datetime(ms['Offense Date'], errors='coerce').dt.date
import math
ms['years_since_offense'] = (reference_date - ms['Offense Date'])/pd.Timedelta(1, 'D')/365

ms['years_since_offense'] = ms['years_since_offense'].fillna(999).astype(int)
ms['years_since_offense'].loc[ms['years_since_offense']<999].describe()

count    343072.000000
mean          3.760619
std           2.443959
min           0.000000
25%           2.000000
50%           4.000000
75%           5.000000
max          69.000000
Name: years_since_offense, dtype: float64

In [5]:
print("The earliest offense date is:    ", min(ms['Offense Date']))
print("The max offense date is:         ", max(ms['Offense Date']), "\n")

print("Distribution of years since offense:", "\n", ms['years_since_offense'].describe(), "\n")

before_2013 = ms['Case Number'][ms['Offense Date']<datetime.date(2013,1,1)].nunique()
before_2014 = ms['Case Number'][ms['Offense Date']<datetime.date(2014,1,1)].nunique()
after_2014 = ms['Case Number'][ms['Offense Date']>=datetime.date(2014,1,1)].nunique()
after_2013 = ms['Case Number'][ms['Offense Date']>=datetime.date(2013,1,1)].nunique()


print("There are", before_2014, "cases with offense date prior to Jan 1, 2014",
     "and", before_2013, "cases before 2013")

print("Percent of cases before 2014:",round(before_2014*100/after_2014,2), "\n"
     "Percent before 2013:", round(before_2013*100/after_2013,2))

The earliest offense date is:     1951-06-30
The max offense date is:          2019-12-30 

Distribution of years since offense: 
 count    343099.000000
mean          3.838939
std           9.160435
min           0.000000
25%           2.000000
50%           4.000000
75%           5.000000
max         999.000000
Name: years_since_offense, dtype: float64 

There are 9965 cases with offense date prior to Jan 1, 2014 and 2963 cases before 2013
Percent of cases before 2014: 6.48 
Percent before 2013: 1.84


In [6]:
#ms.to_csv('../../data/raw/ms.csv', index=False)

# msc -- Charges with value counts

msc = ms['Charge'].value_counts().rename_axis('Charge').reset_index(name='Nvalues')
msc['Charge_noA'] = msc['Charge'].map(lambda x: x.replace('Â',""))
msc['Charge_alnum'] = msc['Charge_noA'].str.replace(r'\W+', '', )

### Datasets with expungement info
These were created from the Suffolk and NW data, in for example [MA_Data-1_Raw.ipynb](https://github.com/codeforboston/clean-slate/blob/master/analyses/notebooks/MA_Data-1_Raw.ipynb). 
This follows a similar process as [MA_Data-2_MergeCharges_alt.ipynb](https://github.com/codeforboston/clean-slate/blob/master/analyses/notebooks/MA_Data-2_MergeCharges_alt.ipynb). 

Prosecution charges detailed is the result of matching the NW and Suffolk charges with each other, and then matching that with the [Master Crime List](https://drive.google.com/file/d/11iD3ilejUW28NE6DdUaUkkp3PoPauhCj/view?usp=sharing), then manually filling in missing expungeabiliyt information, and rationalizing duplicate rows. Duplicate rows occurred because within a chapter and section, some charges may be expungeable while others may not. 


In [7]:
#read in procesuction charges detailed file

PCD = pd.read_csv('../../data/processed/prosecution_charges_detailed.csv', encoding='cp1252') 
PCD.rename(columns={"Expungeable.":"Expungeable"}, inplace=True)
columns = ['Charge', 'Chapter', 'Section', 'Expungeable', 'sex', 'murder', 'extra_criteria']
PCD = PCD[columns]
PCD['Charge_noA'] = PCD['Charge'].map(lambda x: x.replace('Â',""))

PCD['Charge_alnum'] = PCD['Charge_noA'].str.replace(r'\W+', '', )
#PCD.info()

#read in additional info that fills in blanks from PCD. This was manually done by cross-referncing with MCL
# Then we confirmed  with Sana for things that weren't obvious. 

addtl_exp = pd.read_csv('../../data/raw/missing_expungeability_08-02.csv', encoding='cp1252') 
addtl_exp.rename(columns={"Expungeable.":"Expungeable"}, inplace=True)
columns = ['Charge', 'Expungeable', 
           'Reason not expungeable', 'Analysis notes']
addtl_exp = addtl_exp[columns]
addtl_exp['Expungeable'].replace({'yes': 'Yes', 'no': 'No', 'na': 'NA--CMR'}, inplace=True)


# merge these two expungement info datasets

exp = PCD.merge(addtl_exp, on='Charge', how='left')

def replace_values_merge(df, new, x, y):
    df[new] = df[x]
    df[new].fillna(df[y], inplace=True)
    df.drop([x,y], axis=1, inplace=True)
    #print(df[new].value_counts(dropna=False))
    
replace_values_merge(exp, "Expungeable", "Expungeable_x", "Expungeable_y")

exp['CMRoffense'] = False
exp.loc[exp['Charge'].str.contains("CMR"), 'CMRoffense'] = True

exp.loc[(exp['Expungeable'] == "Yes") & (exp['CMRoffense'] == True), ['Expungeable']] = "NotApplicable"

exp.loc[(exp['Charge'] == "BURGLARY, UNARMED & ASSAULT c266 Â§14"),['Expungeable']] = "No" # not expungealbe
exp.loc[(exp['Charge'] == "ATTEMPT TO COMMIT CRIME c274 Â§6"),['Expungeable']] = "Attempt" # need more info 
exp.loc[(exp['Charge'] == "ATTEMPT TO COMMIT CRIME, HABITUAL c274 Â§6"),['Expungeable']] = "Attempt" # need more info 

exp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1650 entries, 0 to 1649
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Charge                  1650 non-null   object
 1   Chapter                 1555 non-null   object
 2   Section                 1562 non-null   object
 3   sex                     1650 non-null   int64 
 4   murder                  1650 non-null   int64 
 5   extra_criteria          41 non-null     object
 6   Charge_noA              1650 non-null   object
 7   Charge_alnum            1650 non-null   object
 8   Reason not expungeable  437 non-null    object
 9   Analysis notes          7 non-null      object
 10  Expungeable             1650 non-null   object
 11  CMRoffense              1650 non-null   bool  
dtypes: bool(1), int64(2), object(9)
memory usage: 156.3+ KB


In [8]:
# Merge data on expungement. Unique offenses. 

ms_merged = msc.merge(exp, on='Charge_alnum', how='left', indicator = True)
print(ms_merged._merge.value_counts())

ms_merged['unmatched'] = ms_merged['_merge']=="left_only"

replace_values_merge(ms_merged, "Charge", "Charge_x", "Charge_y")

ms_merged = ms_merged.drop(columns = ['Charge_noA_x', 'Charge_noA_y',
                                      '_merge'])
ms_merged.info()

both          921
left_only     336
right_only      0
Name: _merge, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1257 entries, 0 to 1256
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Nvalues                 1257 non-null   int64  
 1   Charge_alnum            1257 non-null   object 
 2   Chapter                 903 non-null    object 
 3   Section                 903 non-null    object 
 4   sex                     921 non-null    float64
 5   murder                  921 non-null    float64
 6   extra_criteria          27 non-null     object 
 7   Reason not expungeable  227 non-null    object 
 8   Analysis notes          2 non-null      object 
 9   Expungeable             921 non-null    object 
 10  CMRoffense              921 non-null    object 
 11  unmatched               1257 non-null   bool   
 12  Charge                  1257 non-null   object 
dtypes: bool(1), 

In [9]:
#indicate CMRs
ms_merged.loc[ms_merged['Charge'].str.contains("CMR"), 'CMRoffense'] = 'yes'
ms_merged.CMRoffense.fillna("no", inplace=True)

ms_merged.loc[ms_merged['CMRoffense'] == "yes", ['Expungeable']] = "NA - CMR"

#drop unneeded columns and save list of charges without expungement info
columns = ['Charge', 'Charge_alnum', 'Nvalues', 'Chapter', 'Section', 
           'Expungeable', 'CMRoffense', 'unmatched']

ms_unmatched = ms_merged[columns].loc[ (ms_merged['unmatched']==True) & 
                                      ms_merged['Expungeable'].isnull()]

#ms_unmatched.to_csv('../../data/raw/unmatched_middlesex_to_clean.csv', index=True)

ms_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1257 entries, 0 to 1256
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Nvalues                 1257 non-null   int64  
 1   Charge_alnum            1257 non-null   object 
 2   Chapter                 903 non-null    object 
 3   Section                 903 non-null    object 
 4   sex                     921 non-null    float64
 5   murder                  921 non-null    float64
 6   extra_criteria          27 non-null     object 
 7   Reason not expungeable  227 non-null    object 
 8   Analysis notes          2 non-null      object 
 9   Expungeable             932 non-null    object 
 10  CMRoffense              1257 non-null   object 
 11  unmatched               1257 non-null   bool   
 12  Charge                  1257 non-null   object 
dtypes: bool(1), float64(2), int64(1), object(9)
memory usage: 128.9+ KB


Data exported to csv, saved in the google drive, and farmed out to the team to compare against the Master Crime List to fill in missing expungement information. 

In [10]:
ms_new_data = pd.read_csv('../../data/raw/unmatched_middlesex_clean.csv')

In [11]:
ms_new_data['Charge_alnum2'] = ms_new_data['Charge_alnum'].str.replace("[^a-zA-Z0-9 -]","",)
ms_new_data = ms_new_data.drop(columns = ['Charge', "Charge_alnum", 'Nvalues', 'Group #', 'Name of updater', 'Index'])

ms_merged['Charge_alnum2'] = ms_merged['Charge_alnum'].str.replace("[^a-zA-Z0-9 -]","",)

ms_merged_full = ms_merged.merge(ms_new_data, on='Charge_alnum2', how='outer', indicator=True)

print(ms_merged_full._merge.value_counts(dropna=False))
ms_merged_full.drop(['_merge'], axis=1, inplace=True)


left_only     932
both          325
right_only      0
Name: _merge, dtype: int64


In [12]:
replace_values_merge(ms_merged_full, "Expungeable", "Expungeable_x", "Expungeable_y")
replace_values_merge(ms_merged_full, "Reason not expungeable", "Reason not expungeable_x", "Reason not expungeable_y")

ms_merged_full = ms_merged_full.drop(columns = ['Charge', "Charge_alnum", 'Chapter', 'Section', 'Nvalues', 'CMRoffense'])

In [13]:
ms_merged_charges = ms.merge(ms_merged_full, on='Charge_alnum2', how='outer', indicator=True)
print(ms_merged_charges['_merge'].value_counts())
ms_merged_charges.drop(['_merge'], axis=1, inplace=True)


both          392605
right_only         0
left_only          0
Name: _merge, dtype: int64


### Sex offenses

Some work by Joel: https://colab.research.google.com/drive/1t4WjF6RRSWrf5vtO5WMx4YiBHRRgO6e9#scrollTo=G577BSuLeTjY
Reviewed 'false negatives': many are still not sex offenses as per 100J. 

**not sex offenses**

'NIGHTWALKER, COMMON c272 §53', 
'STREETWALKER, COMMON c272 §53', 
'PROSTITUTION, KEEP HOUSE OF c272 §24',
'BESTIALITY c272 §34', # rule says "engaging in sexual contact with an animal under" c272 s77c, but does not list section 34
'SODOMY c272 §34',
'UNNATURAL ACT c272 §35', # rule lists 35A but not 35. perhaps a different "unnatural act" or minor/not minor
'OBSCENE MATTER, DISTRIBUTE c272 §29', #rule lists 29a but not 29

**additional sex offenses identified by manual review are included in the list below in the code** 

In [14]:
not_sex_offenses = ['NIGHTWALKER, COMMON c272 §53', 
                   'STREETWALKER, COMMON c272 §53', 
                    'PROSTITUTION, KEEP HOUSE OF c272 §24',
                    'BESTIALITY c272 §34', 
                    'SODOMY c272 §34',
                   'UNNATURAL ACT c272 §35', 
                   'OBSCENE MATTER, DISTRIBUTE c272 §29']                   

additional_sex_offenses = ['INDECENT A&B ON CHILD UNDER 14 c265 §13B',
    'PROSTITUTION, PROCURE PERSON TO PRACTICE c272 §12',
    'INDECENT A&B ON CHILD UNDER 14, SUBSQ. c265 §13B',
    'ASSAULT TO RAPE, SUBSQ.OFF. c265 §24',
    'INDECENT A&B ON +60/DISABLED c265 §13H',
    'INDECENT A&B ON CHILD UNDER 14, AGGRAVATED c265 §13B½',
    'INDECENT A&B ON PERSON 14 OR OVER SUBSQ OFFENSE OR AFTER OTHER CERTAIN OFFENSES c265 §13H',
    'INDECENT A&B ON PERSON WITH INTELLECTUAL DISABILITY c265 §13F',
    'PHOTOGRAPH SEXUAL OR INTIMATE PARTS OF CHILD  c272',
    'RAPE OF CHILD WITH FORCE, AGGRAVATED c265 §22B',
    'RAPE OF CHILD WITH FORCE, AGGRAVATED, JOINT ENTERPRISE c265 §22B',
    'RAPE OF CHILD, AGGRAVATED, FIVE YEAR AGE DIFFERENCE c265 §23A',
    'RAPE OF CHILD, AGGRAVATED, TEN YEAR AGE DIFFERENCE c265 §23A',
    'RAPE OF CHILD, STATUTORY, AFTER CERTAIN OFFENSES c265 §23B',
    'RAPE OF CHILD, STATUTORY, AGGRAVATED c265 §23A',
    'SEXUAL CONDUCT WITH CHILD UNDER 18, PAY FOR c272 §53A', #suppressing (c) to avoid an error
    'TRAFFICKING OF PERSON UNDER 18 FOR FORCED SERVICES c265 §51', #suppressing (b) to avoid an error
    'TRAFFICKING OF PERSON UNDER 18 FOR SEXUAL SERVITUDE c265 §50'] ##suppressing (b) to avoid an error

print(ms_merged_charges.sex.value_counts(dropna=False))
for x in additional_sex_offenses:
    ms_merged_charges.loc[ms_merged_charges.Charge.str.contains(x), 'sex'] = 1 
for x in not_sex_offenses:
    ms_merged_charges.loc[ms_merged_charges.Charge.str.contains(x), 'sex'] = 0                            
print(ms_merged_charges.sex.value_counts(dropna=False))

0.0    357277
NaN     32269
1.0      3059
Name: sex, dtype: int64
0.0    356836
NaN     31856
1.0      3913
Name: sex, dtype: int64


In [15]:
sorted(ms_merged_charges['Charge'].loc[(ms_merged_charges['sex'].isnull()) ].unique())

['A&B ATTEMPT WITH FIREARM c265 § 15F',
 'A&B ON CHILD TO JOIN CONSPIRACY c265 §44',
 'A&B ON CORRECTIONAL FACILITY EMPLOYEE c127 §38B(b)',
 'A&B ON FAMILY / HOUSEHOLD MEMBER c265 §13M(a)',
 'A&B ON FAMILY/HOUSEHOLD MEMBER SUBSEQUENT c265 §13M(b)',
 'A&B ON PERSON WITH INTELLECTUAL DISABILITY c265 §13F',
 'A&B WITH BODILY SUBSTANCE ON CORRECTIONAL FACILITY EMPLOYEE c127 §38B(c)',
 'A&B WITH FIREARM c265 § 15E',
 'A&B, SERIOUS BODILY INJURY c265 §13A(b)',
 'ABANDONED/STOLEN MV, IMPROP REMOVE c90 §24H',
 'AGGRAVATED ORGANIZED RETAIL CRIME OVER $10,000 c266 §30D(c)',
 'AID CHILD TO VIOLATE JUVENILE COURT ORDER c119 §63A',
 'ALCOHOL IN MV, POSSESS OPEN CONTAINER OF c90 §24I',
 'AMMUNITION OF UNLAWFUL CALIBER c131 §67',
 'AMMUNITION WITHOUT FID CARD, POSSESS c269$10 (h)(1)',
 'AMMUNITION WITHOUT FID CARD, POSSESS, SUBSQ. OFF. c269 §10(h)(1)',
 'ANIMAL FIGHT, PRESENCE AT c272 §95',
 'ANIMAL INSPECTION, OBSTRUCT DFA c129 §7',
 'ANIMAL QUARANTINE, BREAK c129 §30',
 'ANIMAL, MALICIOUS KILLING O

In [16]:
columns = ['Case Number', 'Offense Date', 'Court Location', 'Charge', 'Charge/Crime Type',
          'Disposition Description', 'CMRoffense', 'Chapter', 'Section', 'Paragraph', 'JuvenileC', 
          'years_since_offense', 'sex', 'murder', 'Expungeable']
ms_merged_charges = ms_merged_charges[columns]

In [17]:
ms_merged_charges.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392605 entries, 0 to 392604
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Case Number              392605 non-null  object 
 1   Offense Date             392578 non-null  object 
 2   Court Location           392605 non-null  object 
 3   Charge                   392605 non-null  object 
 4   Charge/Crime Type        392605 non-null  object 
 5   Disposition Description  392605 non-null  object 
 6   CMRoffense               392605 non-null  bool   
 7   Chapter                  392605 non-null  object 
 8   Section                  389306 non-null  object 
 9   Paragraph                309266 non-null  object 
 10  JuvenileC                392605 non-null  bool   
 11  years_since_offense      392605 non-null  int32  
 12  sex                      360749 non-null  float64
 13  murder                   360336 non-null  float64
 14  Expu

In [18]:
ms_merged_charges.to_csv('../../data/processed/merged_ms.csv', index=False)